<h2 style='color:purple' align='center'>Naive Bayes Tutorial Part 1: Predicting survival from titanic crash</h2>

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [4]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [5]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [6]:
#inputs.Sex = inputs.Sex.map({'male': 1, 'female': 2})

In [7]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [8]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


**I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female**

In [9]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [30]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [10]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [10]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1
3,1,35.0,53.1000,1
4,3,35.0,8.0500,0


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

In [12]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [13]:
model.fit(X_train,y_train)

GaussianNB()

In [14]:
model.score(X_test,y_test)

0.7761194029850746

In [15]:
X_test[0:10]

,Pclass,Age,Fare,female
741,1,36.000000,78.8500,0
551,2,27.000000,26.0000,0
82,3,29.699118,7.7875,1
162,3,26.000000,7.7750,0
611,3,29.699118,7.0500,0
509,3,26.000000,56.4958,0
572,1,36.000000,26.3875,0
867,1,31.000000,50.4958,0
758,3,34.000000,8.0500,0
878,3,29.699118,7.8958,0


In [16]:
y_test[0:10]

741    0
551    0
82     1
162    0
611    0
509    1
572    1
867    0
758    0
878    0
Name: Survived, dtype: int64

In [17]:
model.predict(X_test[0:10])

array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [18]:
model.predict_proba(X_test[:10])

array([[0.45021355, 0.54978645],
       [0.92056104, 0.07943896],
       [0.45070479, 0.54929521],
       [0.96439243, 0.03560757],
       [0.9661656 , 0.0338344 ],
       [0.93602576, 0.06397424],
       [0.76971669, 0.23028331],
       [0.67851936, 0.32148064],
       [0.96788078, 0.03211922],
       [0.9662877 , 0.0337123 ]])

**Calculate the score using cross validation**

In [19]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.768     , 0.792     , 0.728     , 0.78225806, 0.76612903])